In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/binary-class-ins/train.csv/train.csv
/kaggle/input/binary-class-ins/test.csv/test.csv


In [2]:
import tensorflow as tf

2024-07-15 17:56:10.018394: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-15 17:56:10.018584: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-15 17:56:10.181392: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import matplotlib.pyplot as plt

In [4]:
import random

In [5]:
df_train = pd.read_csv('/kaggle/input/binary-class-ins/train.csv/train.csv')
df_test = pd.read_csv('/kaggle/input/binary-class-ins/test.csv/test.csv')
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

gb = df_train.groupby('Response')    
df_yes = gb.get_group(1) 
df_no = gb.get_group(0)

df_samples=df_no.sample(3000000)
df_samples.shape

(3000000, 16)

In [6]:
frames = [df_yes, df_samples]
X_train = pd.concat(frames)
X_train = X_train.sample(frac=1)
X_train = X_train.sort_index()
X_train.head()
X_train.set_index('id', inplace=True)

In [7]:
'''plt.figure(figsize=(6,6))
plt.subplot(1,2,1)
plt.hist(df_yes['Vehicle_Age_1-2 Year'].astype(int))
plt.subplot(1,2,2)
plt.hist(df_no['Vehicle_Age_1-2 Year'].astype(int))

#drop region_code
#drop driving license 
#drop Annual Premium
#drop Vehicle_Age_> 2 Years
#drop Gender Female
#Policy sales Channel, Age, Vehicle Damage,Previously'''

"plt.figure(figsize=(6,6))\nplt.subplot(1,2,1)\nplt.hist(df_yes['Vehicle_Age_1-2 Year'].astype(int))\nplt.subplot(1,2,2)\nplt.hist(df_no['Vehicle_Age_1-2 Year'].astype(int))\n\n#drop region_code\n#drop driving license \n#drop Annual Premium\n#drop Vehicle_Age_> 2 Years\n#drop Gender Female\n#Policy sales Channel, Age, Vehicle Damage,Previously"

In [8]:
y_train = pd.DataFrame(X_train['Response'])

In [9]:
X_train = X_train.drop('Response',axis=1)
X_train = X_train.drop('Driving_License', axis=1)
X_train = X_train.drop('Region_Code', axis=1)
X_train = X_train.drop('Annual_Premium', axis=1)
X_train = X_train.drop('Vehicle_Age_1-2 Year', axis=1)
X_train = X_train.drop('Gender_Female', axis=1)
X_train = X_train.drop('Vehicle_Damage_No', axis=1)

In [10]:
X_test = df_test

In [11]:
X_test = X_test.drop('Driving_License', axis=1)
X_test = X_test.drop('Region_Code', axis=1)
X_test = X_test.drop('Annual_Premium', axis=1)
X_test = X_test.drop('Vehicle_Age_1-2 Year', axis=1)
X_test = X_test.drop('Gender_Female', axis=1)
X_test = X_test.drop('Vehicle_Damage_No', axis=1)

X_test.set_index('id', inplace=True)

In [12]:
X_train["Gender_Male"] = X_train["Gender_Male"].astype(int)
X_train["Vehicle_Age_< 1 Year"] = X_train["Vehicle_Age_< 1 Year"].astype(int)
X_train["Vehicle_Age_> 2 Years"] = X_train["Vehicle_Age_> 2 Years"].astype(int)
X_train["Vehicle_Damage_Yes"] = X_train["Vehicle_Damage_Yes"].astype(int)

In [13]:
X_test["Gender_Male"] = X_test["Gender_Male"].astype(int)
X_test["Vehicle_Age_< 1 Year"] = X_test["Vehicle_Age_< 1 Year"].astype(int)
X_test["Vehicle_Age_> 2 Years"] = X_test["Vehicle_Age_> 2 Years"].astype(int)
X_test["Vehicle_Damage_Yes"] = X_test["Vehicle_Damage_Yes"].astype(int)

In [14]:
X_train['Vintage'] = X_train['Vintage']/X_train['Vintage'].max()
X_train['Policy_Sales_Channel'] = X_train['Policy_Sales_Channel']/X_train['Policy_Sales_Channel'].max()

X_test['Vintage'] = X_test['Vintage']/X_test['Vintage'].max()
X_test['Policy_Sales_Channel'] = X_test['Policy_Sales_Channel']/X_test['Policy_Sales_Channel'].max()

In [15]:
X_val = X_train[3800000:]
X_train = X_train[:3800000]

y_val = y_train[3800000:]
y_train = y_train[:3800000]

In [16]:
y_train_np = y_train['Response'].to_numpy()
y_val_np = y_val['Response'].to_numpy()
X_train_np = X_train.to_numpy()
X_val_np  = X_val.to_numpy()

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10,activation='relu'),
    tf.keras.layers.Dense(10,activation='relu'),
    tf.keras.layers.Dense(10,activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation='sigmoid'),
])

METRICS = [
      #tf.keras.metrics.TruePositives(name='tp'),
      #tf.keras.metrics.FalsePositives(name='fp'),
      #tf.keras.metrics.TrueNegatives(name='tn'),
      #tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      #tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      #tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.01),
    metrics=METRICS
)


epochs=5 
batch_size=1024
history = model.fit(X_train_np, y_train_np, batch_size=batch_size, epochs = epochs)

Epoch 1/5
3711/3711 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.7439 - loss: 0.4574 - prc: 0.5869 - recall: 0.6469
Epoch 2/5
3711/3711 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.7691 - loss: 0.4078 - prc: 0.6329 - recall: 0.7771
Epoch 3/5
3711/3711 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.7705 - loss: 0.4063 - prc: 0.6437 - recall: 0.7693
Epoch 4/5
3711/3711 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.7706 - loss: 0.4060 - prc: 0.6451 - recall: 0.7686
Epoch 5/5
3711/3711 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.7707 - loss: 0.4061 - prc: 0.6454 - recall: 0.7675


In [18]:
X_test_np = X_test.to_numpy()
pred = model.predict(X_test_np)

239684/239684 ━━━━━━━━━━━━━━━━━━━━ 255s 1ms/step


In [19]:
X_test.reset_index(level=0, inplace=True)
X_test.insert(1, "Response", pred)

In [20]:
submission = X_test[['id','Response']].copy()
submission.head()

,id,Response
0,11504798,0.039964
1,11504799,0.578152
2,11504800,0.618816
3,11504801,0.000130
4,11504802,0.071221


In [21]:
submission.to_csv('1stSubmission.csv', encoding='utf-8', index=False)

'from sklearn.svm import LinearSVC\nfrom sklearn.pipeline import make_pipeline\nfrom sklearn.preprocessing import StandardScaler\n\n#X_train_np, y_train_np = make_classification(random_state=0)\n#clf = make_pipeline(StandardScaler(),\n                    #LinearSVC(random_state=0, tol=1e-5))\n    \nclf = make_pipeline(LinearSVC(random_state=0, tol=1e-5))\nclf.fit(X_train_np, y_train_np)'